In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Model

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True,threshold=np.inf)
pd.options.display.max_rows = 999

2023-02-21 11:41:44.367896: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-21 11:41:44.474044: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-21 11:41:44.474061: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-21 11:41:45.004577: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

df = pd.read_csv("kddcup99_csv.csv")
col2encode = ["protocol_type","service","flag"]
ohe_col = pd.get_dummies(df[col2encode],columns=col2encode)
df = df.drop(columns=col2encode)
df_ohe = pd.concat([df,ohe_col],axis=1)

attack_values = df["label"].unique()

label = pd.get_dummies(df_ohe["label"],columns="label")
df_ohe = df_ohe.drop(columns="label",axis=1)

X_train, X_test, y_train, y_test = train_test_split(df_ohe, label, test_size=0.25, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_sc_train = scaler.transform(X_train)
X_sc_test = scaler.transform(X_test)

NCOMPONENTS = 36

pca = PCA(n_components=NCOMPONENTS)
X_pca_train = pca.fit_transform(X_sc_train)
X_pca_test = pca.transform(X_sc_test)

print(X_pca_train.shape)
print(X_pca_test.shape)

X_pca_train  = X_pca_train.reshape(X_pca_train.shape[0],6,6)
X_pca_test  = X_pca_test.reshape(X_pca_test.shape[0],6,6)

print(X_pca_train.shape)
print(X_pca_test.shape)


(370515, 36)
(123505, 36)
(370515, 6, 6)
(123505, 6, 6)


In [3]:

from keras import Input
from keras.regularizers import l1
from keras.layers import MaxPooling2D, Flatten, BatchNormalization, Dense
from qkeras import QConv2DBatchnorm, QActivation, QDense

x = x_in = Input(shape=(6,6,1))

x = QConv2DBatchnorm(32, kernel_size=(3,3), strides=(1,1),
                     kernel_quantizer="quantized_bits(6,0,alpha=1)",
                     bias_quantizer="quantized_bits(6,0,alpha=1)",
                     kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001), use_bias=True,
                     name='fused_convbn')(x)
x = QActivation('quantized_relu(6)',name='conv_act1')(x)
x = MaxPooling2D(pool_size = (2,2),name='pool')(x)
x = Flatten()(x)
x = BatchNormalization(name='bn_dense')(x)
x_out = Dense(len(attack_values),
           name='output_dense')(x)
#x_out = Activation('softmax',name='output_softmax')(x)
model = Model(inputs=[x_in], outputs=[x_out], name='qkeras')

model.summary()

2023-02-21 11:41:54.032354: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-21 11:41:54.032420: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-21 11:41:54.032446: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-21 11:41:54.032472: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-02-21 11:41:54.032498: W tensorfl

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
Model: "qkeras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6, 6, 1)]         0         
                                                                 
 fused_convbn (QConv2DBatchn  (None, 4, 4, 32)         449       
 orm)                                                            
                                                                 
 conv_act1 (QActivation)     (None, 4, 4, 32)          0         
                                                                 
 pool (MaxPooling2D)         (None, 2, 2, 32)          0         
                                                                 
 flatten (Flatten)           (None, 128)               0 

In [6]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics =['accuracy']
)

model.fit(X_pca_train, y_train, batch_size=64, epochs=10,validation_data=(X_pca_test,y_test))

Epoch 1/10
5790/5790 [==============================] - 45s 8ms/step - loss: 14.3144 - accuracy: 0.1213 - val_loss: 14.7645 - val_accuracy: 0.0463
Epoch 2/10
5790/5790 [==============================] - 44s 8ms/step - loss: 14.0869 - accuracy: 0.0900 - val_loss: 14.9240 - val_accuracy: 0.1018
Epoch 3/10
5790/5790 [==============================] - 44s 8ms/step - loss: 14.3950 - accuracy: 0.0978 - val_loss: 14.9377 - val_accuracy: 0.0690
Epoch 4/10
5790/5790 [==============================] - 43s 8ms/step - loss: 9.9905 - accuracy: 0.0684 - val_loss: 5.9474 - val_accuracy: 0.0878
Epoch 5/10
5790/5790 [==============================] - 44s 8ms/step - loss: 8.4367 - accuracy: 0.0896 - val_loss: 4.0896 - val_accuracy: 0.1092
Epoch 6/10
5790/5790 [==============================] - 43s 7ms/step - loss: 6.8704 - accuracy: 0.1111 - val_loss: 13.2505 - val_accuracy: 0.0805
Epoch 7/10
5790/5790 [==============================] - 44s 8ms/step - loss: 8.5118 - accuracy: 0.1514 - val_loss: 15.3804 

In [7]:
print("Evaluate on test data")
results = model.evaluate(X_pca_test, y_test)
print("test acc:", results)
#model.save("model3")


Evaluate on test data
3860/3860 [==============================] - 11s 3ms/step - loss: 13.9963 - accuracy: 0.2750
test acc: [13.99626636505127, 0.2750495970249176]


In [ ]:
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)
co['PruneLowMagnitude'] = pruning_wrapper.PruneLowMagnitude

qmodel  = strip_pruning(model)

In [ ]:
import hls4ml
import plotting
import os

os.environ['PATH'] = '/opt/Xilinx/Vivado/2019.2/bin:' + os.environ['PATH']

# Then the QKeras model
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

hls_config_q = hls4ml.utils.config_from_keras_model(qmodel, granularity='name')
hls_config_q['Model']['ReuseFactor'] = 1
hls_config_q['Model']['Precision'] = 'ap_fixed<6,0>'
  
cfg_q = hls4ml.converters.create_config(backend='Vivado')
cfg_q['IOType']     = 'io_stream' # Must set this if using CNNs!
cfg_q['HLSConfig']  = hls_config_q
cfg_q['KerasModel'] = qmodel
cfg_q['OutputDir']  = 'quantized_pruned_cnn_multi/'
cfg_q['XilinxPart'] = 'xc7z020clg484-1'
  
hls_model_q = hls4ml.converters.keras_to_hls(cfg_q)
hls_model_q.compile()

In [ ]:
hls_model_q.build(csim=False, synth=True, vsynth=True)

In [ ]:
print('Build DONE')